In [1]:
# https://github.com/youngmihuang/lyrics_application
# https://pypi.org/project/GoogleNews/

from GoogleNews import GoogleNews

import requests 
from bs4 import BeautifulSoup
import pandas as pd

googlenews = GoogleNews()

googlenews.setlang('cn')
googlenews.setperiod('d')
googlenews.setencode('utf-8')
googlenews.clear()


x = input("請輸入要搜尋的關鍵字，將為你搜集相關字詞內容:")
googlenews.search(x)

alldata = googlenews.result()
result = googlenews.gettext()
links = googlenews.get_links()
# print(type(result))
# print(len(result))
# print(alldata)


print()

for n in range(len(result)):
#     print(n+1)
    print(result[n])
    print(links[n])
   

df = pd.DataFrame(
{
    '標題': result,
    '連結': links
})
 
df


請輸入要搜尋的關鍵字，將為你搜集相關字詞內容:疫情

沈富雄稱疫情｢小感染｣被內外宣成大災難：從來沒看過民進黨那麼笨
https://newtalk.tw/news/view/2021-05-21/576832
獨家/高雄疫情｜取締3件換1支嘉獎網諷：我開口罩罰單升巡佐｜ 蘋果新聞網｜ 蘋果日報
https://tw.appledaily.com/local/20210521/TX5ND2RBLBA3JMYW7AV6WWTTOM/
陸安徽遼寧疫情 認定來自境外感染
https://udn.com/news/story/121707/5474021
【圖表】中和病例最多嗎？新北疫情累計756例 看各區詳細確診數
https://udn.com/news/story/120940/5474225
疫情升級餐飲急凍 宅配熱起來
https://udn.com/news/story/7266/5474219
獨家｜和平醫院疫情又擴散？ 護師爆料：呼吸治療師也染疫｜ 蘋果新聞網｜ 蘋果日報
https://tw.appledaily.com/life/20210521/QB5QJMQ525BIFKH6YJFV4W4H4Y/
新北疫情 憂院內感染擴散盼全院普篩員工質疑：為何台大行亞東不行｜ 蘋果新聞網｜ 蘋果日報
https://tw.appledaily.com/life/20210521/LEFALTDGEBGA5LTLZYTMPLTMEQ/
疫情非得已！才子神改編庾澄慶經典曲
https://tw.news.yahoo.com/%E7%96%AB%E6%83%85%E9%9D%9E%E5%BE%97%E5%B7%B2-%E6%89%8D%E5%AD%90%E7%A5%9E%E6%94%B9%E7%B7%A8%E5%BA%BE%E6%BE%84%E6%85%B6%E7%B6%93%E5%85%B8%E6%9B%B2-165502268.html
最慘520！疫情凶跳電又缺水蔡英文挑戰歷史定位還要過6大關｜ 蘋果新聞網｜ 蘋果日報
https://tw.appledaily.com/politics/20210520/63RZW3SUCRERXF5ANX3GIYOMIE/
獨家／台中疫情｜應召站野放金絲貓不甩警戒Ｄ奶妹網揪「人與人連結」 ｜ 蘋果新聞網｜ 蘋果日報
https

,標題,連結
0,沈富雄稱疫情｢小感染｣被內外宣成大災難：從來沒看過民進黨那麼笨,https://newtalk.tw/news/view/2021-05-21/576832
1,獨家/高雄疫情｜取締3件換1支嘉獎網諷：我開口罩罰單升巡佐｜ 蘋果新聞網｜ 蘋果日報,https://tw.appledaily.com/local/20210521/TX5ND...
2,陸安徽遼寧疫情 認定來自境外感染,https://udn.com/news/story/121707/5474021
3,【圖表】中和病例最多嗎？新北疫情累計756例 看各區詳細確診數,https://udn.com/news/story/120940/5474225
4,疫情升級餐飲急凍 宅配熱起來,https://udn.com/news/story/7266/5474219
5,獨家｜和平醫院疫情又擴散？ 護師爆料：呼吸治療師也染疫｜ 蘋果新聞網｜ 蘋果日報,https://tw.appledaily.com/life/20210521/QB5QJM...
6,新北疫情 憂院內感染擴散盼全院普篩員工質疑：為何台大行亞東不行｜ 蘋果新聞網｜ 蘋果日報,https://tw.appledaily.com/life/20210521/LEFALT...
7,疫情非得已！才子神改編庾澄慶經典曲,https://tw.news.yahoo.com/%E7%96%AB%E6%83%85%E...
8,最慘520！疫情凶跳電又缺水蔡英文挑戰歷史定位還要過6大關｜ 蘋果新聞網｜ 蘋果日報,https://tw.appledaily.com/politics/20210520/63...
9,獨家／台中疫情｜應召站野放金絲貓不甩警戒Ｄ奶妹網揪「人與人連結」 ｜ 蘋果新聞網｜ 蘋果日報,https://tw.appledaily.com/local/20210521/HCA4H...


In [2]:

url = df['連結'][0]
print(url)


user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}


r = requests.get(url, headers=user_agent)
r.encoding = "utf-8"
web_content = r.text
soup = BeautifulSoup(web_content,'html')


articleContent = soup.find_all('p')
# articleContent

article = []
for p in articleContent:
    article.append(p.text)


articleAll = '\n'.join(article)
# print(articleAll)

# 分段用

https://newtalk.tw/news/view/2021-05-21/576832


In [3]:

import jieba
jieba.load_userdict('dict.txt.big.txt')

d = articleAll.replace('[^\w\s]','').replace('／',"").replace('《','').replace('》','').replace('，','').replace('。','').replace('「','').replace('」','').replace('（','').replace('）','').replace('！','').replace('？','').replace('、','').replace('▲','').replace('…','').replace('：','')
# print(d)

jieba.setLogLevel(20)

# Sentence = jieba.cut(d, cut_all=True)
# print('全模式'+": "  + "/ ".join(Sentence) + '\n')   
 
# Sentence = jieba.cut(d, cut_all=False)
# print('精確模式'+": " + "/ ".join(Sentence)+ '\n')  
 
# Sentence = jieba.cut(d)  
# print('Default為精確模式'+": " + "/ ".join(Sentence)+ '\n')
 
Sentence = jieba.cut_for_search(d)  
print('搜索引擎模式'+": " + "/ ".join(Sentence))
print(Sentence)


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2p/qz_90rqs1fx0_vgnt2xgf6yw0000gn/T/jieba.cache
Loading model cost 0.879 seconds.
Prefix dict has been built successfully.


搜索引擎模式: 目前/ 設定/ 
/ 目前/ 設定/ 
/ 目前/ 設定/ 
/ 台灣/ 爆發/ 武漢/ 肺炎/ 社區/ 感染/ 本土/ 確診/ 案例/ 飆升/ 指揮/ 中心/ 日前/ 緊急/ 宣布/ 全國/ 進入/ 第三/ 級/ 警戒/ 民眾/ 外出/ 時須/ 全程/ 配戴/ 口罩/ 也/ 停止/ 室/ 內/ 5/ 人/ 以上/ 室外/ 10/ 人/ 以上/ 之聚會/ 對/ 此前/ 民進/ 黨/ 立委/ 沈富雄/ 20/ 日/ 在/ 政論/ 節目/ 平論/ 無/ 雙中/ 表示/ 對/ 疫情/ 他/ 是/ 相/ 對/ 較樂觀/ 的/ 他/ 是/ 全國/ 唯一/ 敢告/ 訴/ 大家/ 不要/ 怕/ 免驚/ 這些/ 都/ 是/ 過度/ 反應/ 過分/ 誇張/ 疫情/ 的/ 人/ 而民進/ 黨/ 卻/ 把/ 自己/ 內宣/ 外宣/ 成一個/ 大災難/ 從/ 來/ 沒/ 有/ 看/ 過民/ 進黨/ 這麼/ 笨/ 
/ 沈富雄/ 強調/ 疫情/ 不能/ 看/ 總數/ 總數/ 都/ 在/ 280290/ 例/ 大概/ 已經來/ 到/ 一個/ 高原/ 高原期/ 但是/ 現在/ 這些/ 專家/ 有/ 幾個/ 很/ 有趣/ 的/ 假設/ 他們/ 假設/ 開始/ 時/ 的/ 熱點/ 基礎/ 再生/ 數/ R0/ 值/ 是/ 56/ 若用/ 窩/ 來/ 形容/ 群聚/ 感染/ 在/ 原始/ 的/ 窩/ 比如/ 萬華/ 阿公/ 店/ 或/ 獅子會/ 病毒/ 出/ 了/ 這個/ 窩/ 以/ 後/ R0/ 值馬上/ 下去/ 了/ 會/ 降至/ 1/ 到/ 2/ 左右/ 所以/ 不能/ 用/ 這種/ 老式/ 教科/ 書上/ 的/ 方法/ 去/ 推估/ 沈富雄/ 認為/ 若/ 萬華區/ 所有/ 有人/ 所有人/ 都/ 去/ 普篩/ R0/ 值會/ 在/ 1/ 以下/ 
/ 沈富雄/ 指出/ 現在/ 疫情/ 要/ 看/ 兩點/ 首先/ 是/ 要/ 看/ 萬華板橋/ 中/ 永和/ 的/ 疫情/ 有/ 沒/ 有/ 下去/ 其中/ 萬華/ 阿公/ 店比較/ 麻煩/ 比如/ 板橋/ 人/ 跑/ 到/ 萬華/ 消費/ 後/ 又/ 回到/ 板橋/ 所以/ 比較/ 麻煩/ 再/ 來/ 是/ 要/ 看/ 零星/ 的/ 疫情/ 比如/ 基隆/ 桃園/ 彰化/ 的/ 疫情/ 有/ 沒/ 有/ 下去/ 其中/ 彰化/ 傳

In [6]:
import numpy as np

from PIL import Image

from collections import Counter

import matplotlib.pyplot as plt

from wordcloud import WordCloud, ImageColorGenerator

from scipy.ndimage import gaussian_gradient_magnitude

with open('stopword.txt', 'r', encoding="utf-8") as f:
    stopwords = f.read().split('\n')

terms = {'1','她'}
for sentence in Sentence:
    if sentence in stopwords:
        continue

    if sentence in terms:
        terms[sentence] += 1
    else:
        terms[sentence] = 1

print(Counter(terms))
# generate_from_text()方法會採納stopwords參數 也可使用

# https://coolors.co/palettes/popular


artDf = pd.DataFrame.from_dict(terms, orient='index', columns=['詞頻'])
artDf.sort_values(by=['詞頻'], ascending=False)


img = "color"
img_path = "%s.png" % img

mask_color = np.array(Image.open(img_path))
mask_color = mask_color[::3, ::3]
mask_image = mask_color.copy()
mask_image[mask_image.sum(axis=2) == 0] = 255

edges = np.mean([gaussian_gradient_magnitude(mask_color[:, :, i] / 255., 2) for i in range(3)], axis=0)
mask_image[edges > .08] = 255

wc = WordCloud(font_path="/Users/larry/Library/Fonts/SourceHanSansTW-Regular.otf",
               mask=mask_color,
               max_font_size=45,
               max_words=2000,
               stopwords=stopwords,
               margin=1,
               relative_scaling=0)

wc.generate_from_frequencies(terms)
image_colors = ImageColorGenerator(mask_color)
wc.recolor(color_func=image_colors)

# 視覺化

plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.figure(figsize=(25, 25))
plt.show()

# plt.savefig("Wordcloud.png")
wc.to_file("img/2021-0515.png")


Counter({'1': 1, '她': 1})


AttributeError: 'set' object has no attribute 'values'

In [7]:

artDf = pd.DataFrame.from_dict(dicition, orient='index', columns=['詞頻'])
Frequence = artDf.sort_values(by=['詞頻'], ascending=False).head(50)
print(Frequence)

NameError: name 'dicition' is not defined